In [1]:
import nsepy
from datetime import date
import math
import pandas as pd
import numpy as np
from beakerx.object import beakerx

In [2]:
sym='TCS' #Change this to INFY and NIFTYIT to get desired analysis

## Data Extraction

In [40]:
if sym=='NIFTYIT':
    df=nsepy.get_history(sym,date(2015,1,1),date(2015,12,31),index=True)
else:
    df=nsepy.get_history(sym,date(2015,1,1),date(2015,12,31))
df.to_csv(f'{sys}.csv')

Exception in thread Thread-43:
Traceback (most recent call last):
  File "c:\program files\python36\lib\site-packages\urllib3\connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "c:\program files\python36\lib\site-packages\urllib3\connectionpool.py", line 384, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "c:\program files\python36\lib\site-packages\urllib3\connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse()
  File "c:\program files\python36\lib\http\client.py", line 1331, in getresponse
    response.begin()
  File "c:\program files\python36\lib\http\client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "c:\program files\python36\lib\http\client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "c:\program files\python36\lib\socket.py", line 586, in readinto
    return self._sock.recv_into(b)
  File "c:\p

AttributeError: 'ThreadReturns' object has no attribute 'result'

### Loading Data

In [3]:
df=pd.read_csv(f'{sym}.csv')

## Moving Average

In [4]:
def movingAvg(time_series,window_size):
    # Efficient: time_series.rolling(window_size).mean().shift()
    prevAvg=sum(time_series[:window_size])/window_size
    res=np.array([0.0]*len(time_series))
    res[:window_size]=pd.np.NAN
    res[window_size]=prevAvg
    for i in range(window_size,len(time_series)-1):
        prevAvg+=(time_series[i] - time_series[i-window_size])/window_size
        res[i+1]=prevAvg
    return res

In [5]:
for i in [4,16,26,52]:
    df[f'MA_{i}']=movingAvg(df['Close'],i)

In [6]:
df.head(5)

## Adding Other features

Volume Shocks

In [7]:
def volShock(vols,percent):
    shock=np.array([0]*len(vols))
    polarity=np.array([0]*len(vols))
    for i in range(1,len(vols)):
        per_val=vols[i-1]*(percent/100)
        if abs(vols[i]-vols[i-1])>per_val:
            shock[i]=1
            if vols[i]-vols[i-1]>0:
                polarity[i]=1
    return shock,polarity

In [8]:
df['Volume Shock'],df['Direction VShock']=volShock(df['Volume'],10)

In [9]:
df.head()

Price Shocks

In [10]:
def priceShock(prices,percent):
    shock=np.array([0]*len(vols))
    polarity=np.array([0]*len(vols))
    for i in range(0,len(vols)-1):
        per_val=vols[i+1]*(percent/100)
        if abs(vols[i]-vols[i+1])>per_val:
            shock[i]=1
            if vols[i]-vols[i-1]>0:
                polarity[i]=1
    return shock,polarity

In [11]:
df['Price Shock'],df['Direction PShock']=volShock(df['Close'],2)

In [19]:
df

## Visualization

In [36]:
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
from datetime import datetime

In [37]:
output_notebook()

Loading BokehJS ...

In [35]:
df['Date']=df['Date'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d'))

In [38]:
TOOLS = 'crosshair,save,pan,box_zoom,reset,wheel_zoom'
p = figure(title="Visualization", y_axis_type="linear",x_axis_type='datetime', tools = TOOLS)
p.line(df['Date'],df['Close'],legend='Closing Price')
p.line(df['Date'],df['Volume Shock'],legend='Volume Shock',line_color='red')
show(p)